In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data= pd.read_csv('/content/atis_intents_train.xls',names= ["target", "text"])

In [ ]:
train_data.head()

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [ ]:
test_data= pd.read_csv('/content/atis_intents_test.xls',names= ["target", "text"])

In [ ]:
_LABEL_NAMES = test_data['target']
_LABEL_NAMES = _LABEL_NAMES.unique()
_LABEL_NAMES

array(['atis_flight', 'atis_airfare', 'atis_ground_service',
       'atis_airline', 'atis_flight_time', 'atis_quantity',
       'atis_abbreviation', 'atis_aircraft'], dtype=object)

In [ ]:
train_data= train_data.append(train_data.loc[train_data.target.isin(["atis_flight_time", "atis_quantity"]), :])

In [ ]:
from sklearn.preprocessing import OneHotEncoder as OHE

y_encoder= OHE().fit(np.array(train_data.target).reshape(-1,1))

In [ ]:
ytr_encoded= y_encoder.transform(np.array(train_data.target).reshape(-1,1)).toarray()
yts_encoded= y_encoder.transform(np.array(test_data.target).reshape(-1,1)).toarray()

In [ ]:
import nltk
train_data["lower_text"]= train_data.text.map(lambda x: x.lower())
test_data["lower_text"]= test_data.text.map(lambda x: x.lower())

In [ ]:
from nltk import word_tokenize
nltk.download('punkt')

train_data["tokenized"]= train_data.lower_text.map(word_tokenize)
test_data["tokenized"]= test_data.lower_text.map(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
test_data.head()

,target,text,lower_text,tokenized
0,atis_flight,i would like to find a flight from charlotte ...,i would like to find a flight from charlotte ...,"[i, would, like, to, find, a, flight, from, ch..."
1,atis_airfare,on april first i need a ticket from tacoma to...,on april first i need a ticket from tacoma to...,"[on, april, first, i, need, a, ticket, from, t..."
2,atis_flight,on april first i need a flight going from pho...,on april first i need a flight going from pho...,"[on, april, first, i, need, a, flight, going, ..."
3,atis_flight,i would like a flight traveling one way from ...,i would like a flight traveling one way from ...,"[i, would, like, a, flight, traveling, one, wa..."
4,atis_flight,i would like a flight from orlando to salt la...,i would like a flight from orlando to salt la...,"[i, would, like, a, flight, from, orlando, to,..."


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation

def remove_stop(strings, stop_list):
    classed= [s for s in strings if s not in stop_list]
    return classed

stop= stopwords.words("english")
stop_punc= list(set(punctuation))+ stop

train_data["selected"]= train_data.tokenized.map(lambda df: remove_stop(df, stop_punc))
test_data["selected"]= test_data.tokenized.map(lambda df: remove_stop(df, stop_punc))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
test_data.head()

,target,text,lower_text,tokenized,selected
0,atis_flight,i would like to find a flight from charlotte ...,i would like to find a flight from charlotte ...,"[i, would, like, to, find, a, flight, from, ch...","[would, like, find, flight, charlotte, las, ve..."
1,atis_airfare,on april first i need a ticket from tacoma to...,on april first i need a ticket from tacoma to...,"[on, april, first, i, need, a, ticket, from, t...","[april, first, need, ticket, tacoma, san, jose..."
2,atis_flight,on april first i need a flight going from pho...,on april first i need a flight going from pho...,"[on, april, first, i, need, a, flight, going, ...","[april, first, need, flight, going, phoenix, s..."
3,atis_flight,i would like a flight traveling one way from ...,i would like a flight traveling one way from ...,"[i, would, like, a, flight, traveling, one, wa...","[would, like, flight, traveling, one, way, pho..."
4,atis_flight,i would like a flight from orlando to salt la...,i would like a flight from orlando to salt la...,"[i, would, like, a, flight, from, orlando, to,...","[would, like, flight, orlando, salt, lake, cit..."


In [ ]:
from nltk.stem import PorterStemmer

def normalize(text):
    return " ".join(text)

stemmer= PorterStemmer()

train_data["stemmed"]= train_data.selected.map(lambda xs: [stemmer.stem(x) for x in xs])
train_data["normalized"]= train_data.stemmed.apply(normalize)

test_data["stemmed"]= test_data.selected.map(lambda xs: [stemmer.stem(x) for x in xs])
test_data["normalized"]= test_data.stemmed.apply(normalize)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer= Tokenizer(num_words= 10000)
tokenizer.fit_on_texts(train_data.normalized)

tokenized_train= tokenizer.texts_to_sequences(train_data.normalized)
tokenized_test= tokenizer.texts_to_sequences(test_data.normalized)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_padded= pad_sequences(tokenized_train, maxlen= 20, padding= "pre")
test_padded= pad_sequences(tokenized_test, maxlen= 20, padding= "pre")

In [ ]:
train_padded.shape

(4939, 20)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
encoded_train = [one_hot(d,10000) for d in train_data['normalized']]

In [ ]:
encoded_train[0] , encoded_train[1]

([8447, 1839, 2040, 8369, 5296, 2821, 1426, 4953],
 [1468, 4427, 4218, 5479, 5168, 4953])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded_train = pad_sequences(encoded_train, maxlen=50, padding='post')

In [ ]:
encoded_test = [one_hot(d,10000) for d in test_data['normalized']]
encoded_test = pad_sequences(encoded_test, maxlen=50, padding='post')
encoded_test[1]

array([5825,  549, 4628, 9005, 8901, 7141, 4985, 8368, 7684,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
y_tr = []
y_ts = []
for i in ytr_encoded:
  for j in range(0,8):
    if i[j]==1:
      y_tr.append(j)

for i in yts_encoded:
  for j in range(0,8):
    if i[j]==1:
      y_ts.append(j)
ytr_encoded = np.array(y_tr)
yts_encoded = np.array(y_ts)
print(ytr_encoded.shape , yts_encoded.shape)

(4939,) (800,)


In [ ]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping

#from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
#!pip install keras.optimizers
from tensorflow.keras.optimizers import Adam


model = Sequential()
model.add(Embedding(10000,100,mask_zero=True))
model.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(8,activation='softmax'))
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 64)          42240     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
Total params: 1,054,920
Trainable params: 1,054,920
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(a_train, ytr_encoded, epochs=10, batch_size=128, verbose=1)

Epoch 1/10
39/39 [==============================] - 10s 112ms/step - loss: 1.3895 - accuracy: 0.7238
Epoch 2/10
39/39 [==============================] - 4s 112ms/step - loss: 0.7900 - accuracy: 0.7773
Epoch 3/10
39/39 [==============================] - 4s 111ms/step - loss: 0.5972 - accuracy: 0.7973
Epoch 4/10
39/39 [==============================] - 4s 110ms/step - loss: 0.4810 - accuracy: 0.8564
Epoch 5/10
39/39 [==============================] - 4s 111ms/step - loss: 0.3979 - accuracy: 0.8672
Epoch 6/10
39/39 [==============================] - 4s 111ms/step - loss: 0.3380 - accuracy: 0.8771
Epoch 7/10
39/39 [==============================] - 4s 112ms/step - loss: 0.2690 - accuracy: 0.9111
Epoch 8/10
39/39 [==============================] - 4s 112ms/step - loss: 0.2198 - accuracy: 0.9457
Epoch 9/10
39/39 [==============================] - 4s 113ms/step - loss: 0.1706 - accuracy: 0.9603
Epoch 10/10
39/39 [==============================] - 4s 113ms/step - loss: 0.1367 - accuracy: 0.970

In [ ]:
vocab_size = 10000
batch_size = 128

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vocab_size = 1000
e_dim = 40
n_classes = 8
pre_processing_layer = TextVectorization(max_tokens=vocab_size,
                                         output_sequence_length=20,
                                         name='Notes_preprocessing_layer')

# fit the training set to the layer
pre_processing_layer.adapt(train_data['lower_text'])
# store the vocab
vocab = pre_processing_layer.get_vocabulary()

In [ ]:
def vectorize_text(text):
  return np.array(pre_processing_layer(text))

print("Vectorized review", vectorize_text(train_data['lower_text'][0]))
print("Vectorized review", vectorize_text(train_data['lower_text'][1]))

Vectorized review [ 19  71   2  40   3  10  73 423  86  17  80  16  13  73 619  16   5  36
   0   0]
Vectorized review [ 8  4 27 61  3 20  2 23  6 81 36  0  0  0  0  0  0  0  0  0]


In [ ]:
train_ds = train_data['lower_text'].map(vectorize_text)
test_ds = test_data['lower_text'].map(vectorize_text)

In [ ]:
a_train = vectorize_text(train_data['lower_text'][0])
for sent in train_ds[1:]:
  a_train = np.append(a_train,sent)
a_train = a_train.reshape(4939,20)

In [ ]:
a_test = vectorize_text(test_data['lower_text'][0])
for sent in test_ds[1:]:
  a_test = np.append(a_test,sent)
a_test = a_test.reshape(800,20)

In [ ]:

import tensorflow as tf

embed_layer = model.get_layer('embedding')

# build new model with all layers after embedding layer
new_model = tf.keras.Sequential()
for layer in model.layers[1:]:
    new_model.add(layer)

In [ ]:
sample_vectors = a_test
sample_texts = test_data['lower_text']
sample_labels = yts_encoded

In [ ]:
# select a random index
index = 12
#index = tf.cast(tf.random.uniform(shape=[1],
                                  #minval=sample_vectors.shape[0]),
                #dtype=tf.int8).numpy()[0]
# generate a random sample
sample_text = sample_texts[index]
sample_vector = sample_vectors[index]
sample_label = sample_labels[index]
# get embeddings
sample_embed = embed_layer(sample_vector)
# Create a Baseline vector with zero embeddings
baseline_embed = tf.zeros(shape=tf.shape(sample_embed))
# get preds for sample
sample_preds = model(sample_vectors)[index]
# print the results with color codes
#words = [vocab[i] for i in sample_vector]

In [ ]:
words = [vocab[int(i)] for i in sample_vector]
#sample_vector , sample_text , vocab[3]
words , sample_text

(['flight',
  'on',
  'american',
  'from',
  'miami',
  'to',
  'chicago',
  'arrive',
  'in',
  'chicago',
  'about',
  '5',
  'pm',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ' flight on american from miami to chicago arrive in chicago about 5 pm')

In [ ]:
np.where(sample_label==1)[0][0]
sample_label

array([0., 0., 1., 0., 0., 0., 0., 0.])

In [ ]:
def interpolate_texts(baseline, text, m_steps):

    # Generate m_steps intervals for integral_approximation() below.
    alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps+1)
    # text = tf.cast(text, tf.float32)
    alphas_x = alphas[:, tf.newaxis, tf.newaxis]
    delta = text - baseline
    texts = baseline + alphas_x * delta
    return texts

In [ ]:
n_steps = 50

interpolated_texts = interpolate_texts(baseline_embed,
                                       sample_embed,
                                       n_steps)

In [ ]:
interpolated_texts.shape

TensorShape([51, 20, 100])

In [ ]:
sample_text , sample_label

(' on april first i need a ticket from tacoma to san jose departing before 7 am',
 array([0., 0., 1., 0., 0., 0., 0., 0.]))

In [ ]:
def compute_gradients(t, target_class_idx):

    """ compute the gradient wrt to embedding layer output """

    with tf.GradientTape() as tape:
        tape.watch(t)
        print(np.where(target_class_idx==1)[0][0]+1)
        probs = new_model(t)[:,np.where(target_class_idx==1)[0][0]+1]
    grads = tape.gradient(probs, t)
    return grads
#np.where(target_class_idx==3)[0][0]

In [ ]:
target_label = sample_label
#target_label = 5
path_gradients = compute_gradients(interpolated_texts, target_label)

5


In [ ]:
path_gradients.shape

TensorShape([51, 20, 100])

In [ ]:
sample_label

array([0., 0., 0., 0., 1., 0., 0., 0.])

In [ ]:
# sum the grads of the interpolated vectors
all_grads = tf.reduce_sum(path_gradients, axis=0) / n_steps
# mulitply grads by (input - baseline); baseline is zero vectors
x_grads = tf.math.multiply(all_grads, sample_embed)
# sum all gradients across the embedding dimension
igs = tf.reduce_sum(x_grads, axis=-1).numpy()

In [ ]:
len(igs) , len(test_data['lower_text'][7])

(20, 84)

In [ ]:
def  hlstr(string, color='white'):
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"


def colorize(attrs, cmap='PiYG'):
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    """
    
    cmap_bound = tf.reduce_max(tf.abs(attrs))
    norm = Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)

    # now compute hex values of colors
    colors = list(map(lambda x: rgb2hex(cmap(norm(x))), attrs))
    return colors

In [ ]:
import matplotlib as mpl
from matplotlib.colors import Normalize, rgb2hex
import pandas as pd
from IPython.display import HTML

colors = colorize(igs)

In [ ]:
sample_vector[0]

0

In [ ]:
vocab = []
for sent in train_data['normalized']:
  #print(sent)
  for word in sent.split():
    vocab.append(word)
vocab = list(set(vocab))

In [ ]:
sample_vectora

array([ 44.,  20.,   3., 314.,  18.,   6., 460.,  22.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [ ]:
print(vocab[19] , sample_text)

long  i would like to find a flight from charlotte to las vegas that makes a stop in st. louis


In [ ]:
sample_vector , vocab[35]

(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  35,   1, 100, 175, 141], dtype=int32), 'repres')

In [ ]:
model.evaluate(x=test_padded,y=yts_encoded)

25/25 [==============================] - 2s 21ms/step - loss: 0.3873 - accuracy: 0.8637


[0.38731396198272705, 0.8637499809265137]

In [ ]:
model.predict()

In [ ]:
from sklearn.metrics import classification_report

prediction= y_encoder.inverse_transform(model.predict(a_train))
print(classification_report(train_data.target, prediction))

                     precision    recall  f1-score   support

  atis_abbreviation       0.05      0.05      0.05       147
      atis_aircraft       0.02      0.31      0.03        81
       atis_airfare       0.14      0.24      0.17       423
       atis_airline       0.01      0.01      0.01       157
        atis_flight       0.80      0.19      0.31      3666
   atis_flight_time       0.03      0.13      0.04       108
atis_ground_service       0.01      0.02      0.01       255
      atis_quantity       0.00      0.00      0.00       102

           accuracy                           0.17      4939
          macro avg       0.13      0.12      0.08      4939
       weighted avg       0.61      0.17      0.25      4939



In [ ]:
# print the sample and predictions
#sample_text = 'round trip fares from pittsburgh to philadelphia under 1000 dollars'
print(f"Sample Text: {sample_text}\n")

print("Predictions : ")
for index in tf.argsort(sample_preds,
                        axis=-1, direction='DESCENDING').numpy():
    print(f"\t{_LABEL_NAMES[index]} - {sample_preds[index]*100:0.2f}%")

print(f"\nTrue Label: {_LABEL_NAMES[np.where(sample_label==1)[0][0]]}")
print(f"\nAttributions for Label: {_LABEL_NAMES[np.where(target_label==1)[0][0]]}")
print(f"\nTop 5 Important words: "
      f"{[words[i] for i in tf.argsort(igs, -1, 'DESCENDING')]}\n")
HTML("".join(list(map(hlstr, words, colors))))

Sample Text:  flight on american from miami to chicago arrive in chicago about 5 pm

Predictions : 
	atis_flight - 12.59%
	atis_ground_service - 12.59%
	atis_flight_time - 12.53%
	atis_airfare - 12.53%
	atis_airline - 12.52%
	atis_quantity - 12.46%
	atis_aircraft - 12.41%
	atis_abbreviation - 12.37%

True Label: atis_flight_time

Attributions for Label: atis_flight_time

Top 5 Important words: ['', '', '', '', '', '', '', 'chicago', 'to', 'chicago', 'flight', 'about', 'arrive', 'american', 'on', 'from', 'miami', 'pm', 'in', '5']



In [ ]:
#a_test = vectorize_text(test_data['lower_text'][0])
sample_text = test_data['lower_text'][1]
sample_vector = a_test[1]
sample_label = yts_encoded[1]
#for sent in train_ds[1:]:
  #a_train = np.append(a_train,sent)
#a_train = a_train.reshape(4939,30)

In [ ]:
# print the sample and predictions
#sample_text = 'round trip fares from pittsburgh to philadelphia under 1000 dollars'
print(f"Sample Text: {sample_text}\n")

print("Predictions : ")
for index in tf.argsort(sample_preds,
                        axis=-1, direction='DESCENDING').numpy():
    print(f"\t{_LABEL_NAMES[index]} - {sample_preds[index]*100:0.2f}%")

print(f"\nTrue Label: {_LABEL_NAMES[np.where(sample_label==1)[0][0]]}")
print(f"\nAttributions for Label: {_LABEL_NAMES[np.where(target_label==1)[0][0]]}")
print(f"\nTop 5 Important words: "
      f"{[words[i] for i in tf.argsort(igs, -1, 'DESCENDING')]}\n")
HTML("".join(list(map(hlstr, words, colors))))

Sample Text:  i would like to find a flight from charlotte to las vegas that makes a stop in st. louis

Predictions : 
	atis_flight_time - 99.74%
	atis_airline - 0.12%
	atis_quantity - 0.04%
	atis_ground_service - 0.04%
	atis_airfare - 0.03%
	atis_abbreviation - 0.01%
	atis_flight - 0.00%
	atis_aircraft - 0.00%

True Label: atis_flight_time

Attributions for Label: atis_flight_time

Top 5 Important words: ['find', 'would', 'in', 'like', '', 'that', 'louis', 'a', 'st', 'stop', 'from', 'a', 'makes', 'vegas', 'las', 'charlotte', 'to', 'i', 'to', 'flight']

